In [1]:
from pynq import Overlay
from pynq import allocate
import numpy as np

In [2]:
# Program bitstream to FPGA
overlay = Overlay('/home/xilinx/workspace/nn.bit')

In [3]:
# Access to AXI DMA
dma = overlay.axi_dma_0
dma_send = overlay.axi_dma_0.sendchannel
dma_recv = overlay.axi_dma_0.recvchannel

In [4]:
# Allocate physical memory for AXI DMA MM2S
input_buffer = allocate(shape=(14,), dtype=np.uint32)

In [5]:
# Weight and bias
input_buffer[0] = 0x057A057A
input_buffer[1] = 0x0000B07A

input_buffer[2] = 0x03E10314
input_buffer[3] = 0x0000FC66

input_buffer[4] = 0x028F0433
input_buffer[5] = 0x0000FC70

input_buffer[6] = 0xFAC21C70
input_buffer[7] = 0xF5A30051

input_buffer[8] = 0x0685E399
input_buffer[9] = 0x00CC07E1

# Input
input_buffer[10] = 0x20002000
input_buffer[11] = 0x14001400

input_buffer[12] = 0x14002000
input_buffer[13] = 0x14002000

In [6]:
# Check the written data
for i in range(14):
    print("0x%08X" % (input_buffer[i]))

0x057A057A
0x0000B07A
0x03E10314
0x0000FC66
0x028F0433
0x0000FC70
0xFAC21C70
0xF5A30051
0x0685E399
0x00CC07E1
0x20002000
0x14001400
0x14002000
0x14002000


In [7]:
# Do AXI DMA MM2S transfer
dma_send.transfer(input_buffer)

In [8]:
# Allocate physical memory for AXI DMA S2MM
output_buffer = allocate(shape=(4,), dtype=np.uint32)

In [9]:
# Check the memory content
for i in range(4):
    print("0x%08X" % (output_buffer[i]))

0x00000000
0x00000000
0x00000000
0x00000000


In [10]:
# Do AXI DMA S2MM transfer
dma_recv.transfer(output_buffer)

In [11]:
# Check the memory content after DMA transfer
for i in range(4):
    print("0x%08X" % (output_buffer[i]))

0x002B03A8
0x0014002B
0x03CF004D
0x03E803CF


In [12]:
# Output
pred_hw = np.zeros((2,4))

pred_hw[0][0] = ((output_buffer[0] & 0x0000FFFF) / 1024.0)
pred_hw[0][1] = (((output_buffer[0] & 0xFFFF0000) >> 16) / 1024.0)
pred_hw[0][2] = ((output_buffer[1] & 0x0000FFFF) / 1024.0)
pred_hw[0][3] = (((output_buffer[1] & 0xFFFF0000) >> 16) / 1024.0)

pred_hw[1][0] = ((output_buffer[2] & 0x0000FFFF) / 1024.0)
pred_hw[1][1] = (((output_buffer[2] & 0xFFFF0000) >> 16) / 1024.0)
pred_hw[1][2] = ((output_buffer[3] & 0x0000FFFF) / 1024.0)
pred_hw[1][3] = (((output_buffer[3] & 0xFFFF0000) >> 16) / 1024.0)

In [13]:
# Prediction
pred_hw

array([[0.9140625 , 0.04199219, 0.04199219, 0.01953125],
       [0.07519531, 0.95214844, 0.95214844, 0.9765625 ]])

In [14]:
# Rounding
np.round(pred_hw)

array([[1., 0., 0., 0.],
       [0., 1., 1., 1.]])

In [15]:
# Delete buffer to prevent memory leak
del input_buffer, output_buffer